In [5]:
import numpy as np
import h5py
# from nbfinder import NotebookFinder
import sys
# sys.meta_path.append(NotebookFinder())
from util import add_pulse_to_inp_tensor, get_nonempty_pulses, total_doms, total_height, total_width, get_pulse_array, get_nonempty_events

In [6]:

def make_dataset(filename, sig_or_bg):
    
    hf = h5py.File(filename,'r')
    pulse_array_keys = get_nonempty_pulses(hf)
    num_events = len(pulse_array_keys)
    
    ####### Added by Venkitesh, Oct 24, 2018. 
    ## Extracting the weights
    event_array_keys=get_nonempty_events(hf)
    assert len(pulse_array_keys)==len(event_array_keys), "Pulse and event array keys have different sizes"
    # Computing the weights
    wgts=np.array([hf['events'][event_key]['weight'][0] for event_key in event_array_keys])
    
    # Checking whether the event_array_keys and pulse_array_keys are in order and identical
#     print(np.array_equal(pulse_array_keys,event_array_keys))    
    assert np.array_equal(pulse_array_keys,event_array_keys), "Pulse array %s and Event array %s are not identical. Possibility of mismatch"%(pulse_array_keys,event_array_keys)
    #######
        
    tens = np.zeros((num_events, total_doms, total_height, total_width))
    
    for ex_num, pulse_array_key in enumerate(pulse_array_keys):
        pulse_array = get_pulse_array(hf, pulse_array_key)
        add_pulse_to_inp_tensor(tens, ex_num, pulse_array)
        
    lbls = np.ones((num_events,)) if sig_or_bg == "sig" else np.zeros((num_events,))
        
    return tens, lbls, wgts


def get_data(sig_filename_list, bg_filename_list):
    
    ### Changes made by Venkitesh, 10/24/2018
    x, y, wt = make_dataset(sig_filename_list[0], "sig")
    
    for fn in sig_filename_list[1:]:
    #for fn in sig_filename_list:
        xs,ys,wts = make_dataset(fn, "sig")
        x = np.vstack((x,xs))
        y = np.concatenate((y,ys))
        wt = np.concatenate((wt,wts))

    for fn in bg_filename_list:
        xb,yb,wtb = make_dataset(fn, "bg")
        x = np.vstack((x,xb))
        y = np.concatenate((y,yb))
        wt= np.concatenate((wt,wtb))
    return x,y,wt


In [9]:
!jupyter nbconvert --to script load_data.ipynb


[NbConvertApp] Converting notebook load_data.ipynb to script
[NbConvertApp] Writing 2528 bytes to load_data.py


In [8]:
# import glob

# sigpath = "/project/projectdirs/dasrepo/icecube_data/hdf5_out/filtered/nugen/11374/clsim-base-4.0.3.0.99_eff/"
# sig_list=glob.glob(sigpath+'*00.hdf5')
# bgpath = "/project/projectdirs/dasrepo/icecube_data/hdf5_out/filtered/corsika/11057/"
# bg_list=glob.glob(bgpath+'*00.hdf5')

# inx,inpy,wts = get_data(sig_list, bg_list)

